# UNSW-NB15: Preparação da Data

## Carregamento das bibliotecas

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pickle
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Carregamento dos datasets
train = pd.read_csv('./train_alldata_preparacao.csv')
test = pd.read_csv('./test_alldata_preparacao.csv')

In [3]:
# Retirando a coluna 'label' que não terá relevancia nesta parte do problema

train = train.drop(columns='label')
test = test.drop(columns='label')

print(train.columns)
print()
print(test.columns)
print()

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl',
       'dload', 'dloss',
       ...
       'state_CLO', 'state_CON', 'state_ECO', 'state_FIN', 'state_INT',
       'state_PAR', 'state_REQ', 'state_RST', 'state_URN', 'state_no'],
      dtype='object', length=178)

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl',
       'dload', 'dloss',
       ...
       'state_CLO', 'state_CON', 'state_ECO', 'state_FIN', 'state_INT',
       'state_PAR', 'state_REQ', 'state_RST', 'state_URN', 'state_no'],
      dtype='object', length=178)



In [4]:
# Funções uteis
def multi_corr(col1, col2="attack_cat", df=train):
    '''
    CORRELAÇAO + LOG CORRELAÇAO
    A função calcula o coeficiente de correlação de Pearson entre col1 e col2 usando o método corr() e guarda o valor numa variável chamada corr.
    A função então aplica a função np.log1p aos valores em col1.
        A função np.log1p() é usada para calcular o logaritmo natural [1 + col1].
        [A transformação de log pode ser útil para reduzir
        o impacto de outliers ou valores extremos nos dados
        e pode tornar a relação entre as duas variáveis mais
        linear ou mais fácil de modelar.] 
    Calcula a correlação entre col1 transformada e col2 e guarda o valor numa variável chamada log_corr.
    Por fim, a função imprime os valores de corr e log_corr.
    '''

    if df[col2].dtype == 'object':
        dummy_df = pd.get_dummies(df[col2])
        corr = df[col1].corr(dummy_df.iloc[:, 0])
        log_corr = df[col1].apply(np.log1p).corr(dummy_df.iloc[:, 0])
    else:
        corr = df[col1].corr(df[col2])
        log_corr = df[col1].apply(np.log1p).corr(df[col2])

    print("Correlation: {}\nlog_Correlation: {}".format(corr, log_corr))

In [5]:
def corr(col1, col2="attack_cat", df=train):
    """
    CORRELAÇAO
    A função calcula o coeficiente de correlação de Pearson entre col1 e col2 e guarda o valor numa variável chamada corr.
    Devolve o valor de corr.
    """
    return df[[col1, col2]].corr().iloc[0,1]

## Remoção das colunas com correlações elevadas

In [6]:
# Seleção das colunas com correlações elevadas relativemante a outras
# Referência: https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
corr_matrix = train.corr().abs()

# Selecionamos apenas o triangulo superior da matrix de correlação
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Encontra o index das colunas com uma correlação maior que 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [7]:
# Não queremos usar estas colunas nos gráficos porque têm alta corr
# E provavelmente têm o mesmo tipo de comportamento que uma coluna já traçada num gráfico
print(to_drop)

['dbytes', 'dloss', 'state_FIN']


In [8]:
# Irá ser mantida a coluna 'state_FIN', pois o seu comportamento está relacionado com as colunas 'state' e será util na construção dos classificadores

to_drop.remove('state_FIN')
print(to_drop)

['dbytes', 'dloss']


In [9]:
# Irão ser removidas as colunas do dataset de treino e teste

train.drop(columns=to_drop, inplace=True)
test.drop(columns=to_drop, inplace=True)

In [10]:
train.shape, test.shape

((238682, 176), (82332, 176))

## Aplicar log1p nas Colunas numéricas

        A função np.log1p() é usada para calcular o log [1 + col1].
        [A transformação de log pode ser útil para reduzir
        o impacto de outliers ou valores extremos nos dados
        e pode tornar a relação entre as duas variáveis mais
        linear ou mais fácil de modelar.] 

Durante  Parte 2, descobrimos que poucas colunas numéricas mostram uma melhor visualização para curvas PDF((Probability Density Function)) se aplicarmos log1p às colunas.

Então, pensei em tentar log1p em todas as colunas e verificar o valor de correlação da coluna original e da coluna log1p com a coluna de alvo ( "attack_cat")

In [11]:
# Obtendo o número de valores únicos de todas as colunas
# Se os valores únicos forem altos, isso significa que tem um conjunto contínuo de valores (indicador de coluna numérica)
col_unique_values = train.nunique()

In [12]:
# Se os valores unicos forem maiores que um determinado limite, iremos verificar a sua correção
col = col_unique_values[col_unique_values>200].index
print(col)

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'rate', 'dload', 'sinpkt', 'dinpkt',
       'djit', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean'],
      dtype='object')


In [13]:
# Verificando o valor de correlação da coluna original e da coluna com a função log1p aplicada
# Apenas iremos verificar as colunas cujos valores únicos são maiores do que o limite estabelecido, isto é, iremos verificar as colunas numéricas contínuas
columns_with_better_log_corr = []
for column in col:
    print("{:-^30}".format(column))
    result = multi_corr(column)
    if result is not None:
        columns_with_better_log_corr.append(result)

log1p_col = columns_with_better_log_corr

print("Colunas com melhor correlação log: {}".format(columns_with_better_log_corr))

-------------dur--------------
Correlation: 0.008193850308501812
log_Correlation: 0.0001285965917095171
------------spkts-------------
Correlation: -0.012804446890055892
log_Correlation: -0.047091144063490666
------------dpkts-------------
Correlation: -0.017951177352056494
log_Correlation: -0.05841600107740606
------------sbytes------------
Correlation: -0.004016021454124268
log_Correlation: -0.035946752584340824
-------------rate-------------
Correlation: 0.04062753884370234
log_Correlation: 0.03677682299152361
------------dload-------------
Correlation: -0.0321689177222789
log_Correlation: -0.06782275419617483
------------sinpkt------------
Correlation: -0.0133462449080144
log_Correlation: -0.01191193931656315
------------dinpkt------------
Correlation: 0.0064312124290788674
log_Correlation: -0.008382804570139018
-------------djit-------------
Correlation: -0.004628250455907875
log_Correlation: -0.0285850541496051
------------tcprtt------------
Correlation: 0.007317502463395031
log_

In [14]:
def log1p_transform(col, df=train):
    '''
    Esta função aplica a função np.log1p() a uma determinada coluna de um dataframe. 
    De seguida, cria uma nova coluna com o nome da coluna original anexado com _log1p e guarda os valores transformados em log nessa nova coluna.
    Por fim, remove a coluna original do DataFrame.
    '''
    new_col = col+'_log1p'
    df[new_col] = df[col].apply(np.log1p)
    df.drop(col, axis=1, inplace=True)

In [15]:
# Transformando as colunas numéricas com log1p nos conjuntos treino e teste
for col in log1p_col:
    log1p_transform(col, df=train)
    log1p_transform(col, df=test)

print(train.shape, test.shape)
print()
print(train.columns)
print()
print(test.columns)

(238682, 176) (82332, 176)

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'rate', 'sttl', 'dttl', 'dload',
       'sinpkt', 'dinpkt',
       ...
       'state_CLO', 'state_CON', 'state_ECO', 'state_FIN', 'state_INT',
       'state_PAR', 'state_REQ', 'state_RST', 'state_URN', 'state_no'],
      dtype='object', length=176)

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'rate', 'sttl', 'dttl', 'dload',
       'sinpkt', 'dinpkt',
       ...
       'state_CLO', 'state_CON', 'state_ECO', 'state_FIN', 'state_INT',
       'state_PAR', 'state_REQ', 'state_RST', 'state_URN', 'state_no'],
      dtype='object', length=176)


In [16]:
# Separação dos conjuntos de dados de treino e teste em variáveis ​​de recursos (x_train, x_test) e de destino (y_train, y_test).
# A função drop é usada para remover a coluna de rótulo/alvo dos conjuntos de dados e armazená-los em y_train e y_test, enquanto as colunas restantes são armazenadas em x_train e x_test.

x_train, y_train = train.drop(columns=['attack_cat']), train['attack_cat']
x_test, y_test = test.drop(columns=['attack_cat']), test['attack_cat']

In [17]:
print(x_train.shape, y_train.shape)
print()
print(x_test.shape, y_test.shape)

(238682, 175) (238682,)

(82332, 175) (82332,)


In [18]:
x_train.head()

dur  spkts  dpkts  sbytes           rate  sttl  dttl         dload  \
0  1.009410     16     12    2128      26.748298   254   252   4929.612305   
1  0.000006      2      0     136  166666.660800   254     0      0.000000   
2  0.000008      2      0     200  125000.000300   254     0      0.000000   
3  0.000005      2      0     114  200000.005100   254     0      0.000000   
4  0.218607     10      8     776      77.765124    62   252  40620.839840   

      sinpkt     dinpkt  ...  state_CLO  state_CON  state_ECO  state_FIN  \
0  67.293998  85.370453  ...        0.0        0.0        0.0        1.0   
1   0.006000   0.000000  ...        0.0        0.0        0.0        0.0   
2   0.008000   0.000000  ...        0.0        0.0        0.0        0.0   
3   0.005000   0.000000  ...        0.0        0.0        0.0        0.0   
4  24.289667  29.113572  ...        0.0        0.0        0.0        1.0   

   state_INT  state_PAR  state_REQ  state_RST  state_URN  state_no  
0        0.0        0.0        0.0        0.0        0.0       0.0  
1        1.0        0.0        0.0        0.0        0.0       0.0  
2        1.0        0.0        0.0        0.0        0.0       0.0  
3        1.0        0.0        0.0        0.0        0.0       0.0  
4        0.0        0.0        0.0        0.0        0.0       0.0  

[5 rows x 175 columns]

## Normalização Z-Score / Standardizing
    
Como vimos, o intervalo de valores de algumas colunas neste conjunto de dados é muito grande. <br>
Assim sendo, vamos colocar tudo num determinado intervalo usando standardscaler. Depois deste processo, todos os recursos terão média 0 e um desvio padrão de 1

In [19]:
col_unique_values = x_train.nunique()
num_col = col_unique_values[col_unique_values>200].index
print(num_col)

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'rate', 'dload', 'sinpkt', 'dinpkt',
       'djit', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean'],
      dtype='object')


In [20]:
# Aplicar a normalização
scaler = StandardScaler()
scaler = scaler.fit(x_train[num_col])   # ajustar o scaler às colunas numéricas (col) de x_train

x_train[num_col] = scaler.transform(x_train[num_col])   # transforma as colunas numéricas de x_train usando o scaler
x_test[num_col] = scaler.transform(x_test[num_col])   # transforma as colunas numéricas de x_train usando o scaler

x_train.head()

dur     spkts     dpkts    sbytes      rate  sttl  dttl     dload  \
0 -0.041419 -0.057536 -0.108786 -0.036362 -0.490356   254   252 -0.349299   
1 -0.207741 -0.173298 -0.219135 -0.049649  0.619337   254     0 -0.350957   
2 -0.207741 -0.173298 -0.219135 -0.049222  0.341870   254     0 -0.350957   
3 -0.207741 -0.173298 -0.219135 -0.049795  0.841312   254     0 -0.350957   
4 -0.171721 -0.107148 -0.145569 -0.045380 -0.490016    62   252 -0.337294   

     sinpkt    dinpkt  ...  state_CLO  state_CON  state_ECO  state_FIN  \
0 -0.158046 -0.008774  ...        0.0        0.0        0.0        1.0   
1 -0.165495 -0.084236  ...        0.0        0.0        0.0        0.0   
2 -0.165495 -0.084236  ...        0.0        0.0        0.0        0.0   
3 -0.165496 -0.084236  ...        0.0        0.0        0.0        0.0   
4 -0.162807 -0.058501  ...        0.0        0.0        0.0        1.0   

   state_INT  state_PAR  state_REQ  state_RST  state_URN  state_no  
0        0.0        0.0        0.0        0.0        0.0       0.0  
1        1.0        0.0        0.0        0.0        0.0       0.0  
2        1.0        0.0        0.0        0.0        0.0       0.0  
3        1.0        0.0        0.0        0.0        0.0       0.0  
4        0.0        0.0        0.0        0.0        0.0       0.0  

[5 rows x 175 columns]

In [21]:
file_path = 'dataPronta/'

In [22]:
# Verificar se as colunas de treino e teste são as mesmas
all(x_train.columns == x_test.columns)

True

In [23]:
# Guardar todos os dados para usar no treino e teste das redes neuronais
pickle.dump((x_train, y_train), open('./dataPronta/final_train.pkl', 'wb'))
pickle.dump((x_test, y_test), open('./dataPronta/final_test.pkl', 'wb'))

In [24]:
x_test.shape, y_test.shape, x_train.shape, y_train.shape

((82332, 175), (82332,), (238682, 175), (238682,))